# Functions

In [ ]:
# Functions
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import numpy as np
import holidays
import re
from sklearn.preprocessing import MinMaxScaler
from feature_engine.timeseries.forecasting import (
    WindowFeatures,
    ExpandingWindowFeatures,
    LagFeatures
)
import pandas as pd
from feature_engine.datetime import DatetimeFeatures
from feature_engine.creation import CyclicalFeatures
from sktime.transformations.series.fourier import FourierFeatures
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import numpy as np
import holidays
import re
from sklearn.preprocessing import MinMaxScaler
from feature_engine.timeseries.forecasting import (
    WindowFeatures,
    ExpandingWindowFeatures,
    LagFeatures
)
import pandas as pd
from feature_engine.datetime import DatetimeFeatures
from feature_engine.creation import CyclicalFeatures
from sktime.transformations.series.fourier import FourierFeatures

def plot_errors(ErrorSeries):
    """
    Plots the Mean Squared Error (MSE) values against features to visualize feature selection performance.

    Args:
        ErrorSeries (pd.Series): A pandas Series where indices are feature names
                                 and values are corresponding errors (e.g., MSE).
    """
    import matplotlib.pyplot as plt
    import numpy as np

    # Prepare the data
    x = np.arange(len(ErrorSeries))  # Numerical x-axis indices
    y = ErrorSeries.values           # Error values
    labels = ErrorSeries.index       # Feature names

    # Initialize the plot
    plt.figure(figsize=(14, 6))
    plt.style.use("seaborn-v0_8-whitegrid")  # Use clean seaborn grid style

    # Plot the error series
    plt.plot(x, y, marker='o', linestyle='-', color='b', linewidth=2)

    # Customize x-ticks and labels
    plt.xticks(x, labels, rotation=45, ha="right")
    plt.ylabel("MSE (Mean Squared Error)", wrap=True)
    plt.xlabel("Features", wrap=True)
    plt.title("Feature Selection Performance: MSE vs Features", fontsize=14, pad=15)

    # Adjust margins and spacing
    plt.tight_layout()
    plt.show()

# Feature selection using SHAP-ordered features and TS Cross-Validation
def FeatureSelection_fixed_test(regressor, DF_features, DF_target, ordered_features_list, test_size=672, tolerance=50):
    feature_list = []
    error_list = []
    total_samples = len(DF_features)
    n_splits = 5
    no_improvement_count = 0
    min_error = float('inf')

    for i in ordered_features_list:
        feature_list.append(i)
        X = DF_features[feature_list].to_numpy()
        y = DF_target.to_numpy()

        # Custom TS CV logic
        splits = []
        start_train_size = total_samples - (n_splits * test_size)
        for split in range(n_splits):
            train_end = start_train_size + split * test_size
            test_start = train_end
            test_end = test_start + test_size
            if test_end <= total_samples:
                splits.append((list(range(0, train_end)), list(range(test_start, test_end))))

        # Perform cross-validation
        TimeSeriesCVerror = []
        for train_index, test_index in splits:
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            regressor.fit(X_train, y_train)
            predicted_val = regressor.predict(X_test)
            Error = mean_squared_error(y_test, predicted_val)
            TimeSeriesCVerror.append(Error)

        # Average error across splits
        TS_CV_error = sum(TimeSeriesCVerror) / len(TimeSeriesCVerror)
        error_list.append(TS_CV_error)

        # Track improvement
        if TS_CV_error < min_error:
            min_error = TS_CV_error
            no_improvement_count = 0
        else:
            no_improvement_count += 1

        if no_improvement_count >= tolerance:
            print(f"No improvement after {tolerance} features. Stopping early.")
            break

    ErrorSeries = pd.Series(error_list, index=feature_list)
# Plot the errors using a custom plot function
    plot_errors(ErrorSeries)


    return ErrorSeries

def keep_indices_till_min(series):
    selected_features = []
    min_value = series.min()
    for idx, value in series.items():
        if idx not in selected_features:
            selected_features.append(idx)
        if value == min_value:
            break
    return selected_features

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

def peak_RMSE(y_true: np.ndarray, y_pred: np.ndarray, threshold: float = 5000) -> float:
    """
    Calculate RMSE for values where the true values exceed a certain threshold.

    Args:
        y_true (np.ndarray): Array of true target values.
        y_pred (np.ndarray): Array of predicted target values.
        threshold (float): Threshold for filtering peak values (default is 5000).

    Returns:
        float: RMSE for the filtered values.
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    filtered_indices = y_true > threshold

    # Filter both predictions and true values based on the threshold
    filtered_predictions = y_pred[filtered_indices]
    filtered_real_values = y_true[filtered_indices]

    # Avoid division by zero or empty slices
    if len(filtered_real_values) == 0:
        return 0.0

    # Calculate RMSE for the filtered data
    rmse = np.sqrt(mean_squared_error(filtered_real_values, filtered_predictions))
    return rmse

def ErrorCalculator(name: str, y_true: np.ndarray, y_pred: np.ndarray, threshold: float = 5000) -> dict:
    """
    Calculate multiple error metrics for predictions.

    Args:
        name (str): Name of the pipeline or model.
        y_true (np.ndarray): Array of true target values.
        y_pred (np.ndarray): Array of predicted target values.
        threshold (float): Threshold for peak RMSE calculation (default is 5000).

    Returns:
        dict: A dictionary containing RMSE, MAE, MSE, and PRMSE.
    """
    errors = {
        "Pipelines": name,
        "RMSE": np.sqrt(mean_squared_error(y_true, y_pred)),
        "MAE": mean_absolute_error(y_true, y_pred),
        "MSE": mean_squared_error(y_true, y_pred),
        "PRMSE": peak_RMSE(y_true, y_pred, threshold),
    }
    return errors


def CreateWorkHourFeature(input_data):
    """
    Receives as input a DataFrame or Series and outputs a DataFrame with the working hours during the day.
    When the day of the week is larger than 4, it is considered a weekend (1), otherwise, it's a workday (0).
    During workdays and between 8:00 and 17:00, it is considered a working hour.

    Parameters:
    input_data (DataFrame or Series): Input data with a DatetimeIndex.

    Returns:
    DataFrame: DataFrame with the added "WorkingHour_flag" column.
    """
    if isinstance(input_data, pd.Series):
        input_df = pd.DataFrame(input_data)
    elif isinstance(input_data, pd.DataFrame):
        input_df = input_data
    else:
        raise ValueError("Input must be a DataFrame or Series.")

    assert isinstance(input_df.index, pd.DatetimeIndex), "Index must be a datetime index."

    input_df["dayOfWeek"] = input_df.index.dayofweek
    input_df.loc[input_df["dayOfWeek"] > 4, "weekendFlag"] = 1
    input_df.loc[input_df["dayOfWeek"] < 5, "weekendFlag"] = 0
    input_df["hour"] = input_df.index.hour
    input_df["WorkingHour_flag"] = 0
    input_df.loc[((input_df["hour"] > 8) & (input_df["hour"] < 17) & (input_df["weekendFlag"] == 0)), "WorkingHour_flag"] = 1
    input_df.drop(["hour", "dayOfWeek", "weekendFlag"], axis=1, inplace=True)

    return input_df


def ListCreatorFlagger(df, substrings=['flag', 'cos', 'sin','day_of_week', 'day_of_month', 'weekend', 'days_in_month', 'hour', 'minute']):
    """
    A function that separates the columns containing specified substrings from those that don't.
    df is the dataframe in question and the substring is a list.
    """
    flag_columns = [col for col in df.columns if any(substring in col for substring in substrings)]

    if not flag_columns:
        print("No columns with the specified substrings found.")
        return None, None

    non_flag_columns = [col for col in df.columns if col not in flag_columns]

    return non_flag_columns, flag_columns

def HolidayFeatureCreator(input_data):
    """
    Receives as input a DataFrame or Series and creates a column named "Holidays_flag" with 1 if there is a holiday and with 0 if no holidays exist.
    Holidays derived from Germany.
    """
    if isinstance(input_data, pd.Series):
        input_df = pd.DataFrame(input_data)
    elif isinstance(input_data, pd.DataFrame):
        input_df = input_data
    else:
        raise ValueError("Input must be a DataFrame or Series.")

    assert isinstance(input_df.index, pd.DatetimeIndex), "Index must be a datetime index."

    national_holidays_all = holidays.DE(years=[2014,2015,2016,2017,2018,2019,2020, 2021, 2022, 2023, 2024, 2025, 2026]).items()
    national_holidays = [items[0] for items in national_holidays_all]  # this is a list

    # Create a new column for holidays flag
    input_df["Holidays_flag"] = 0

    # Iterate over the index and set holiday flag to 1 if the date matches any national holiday
    for index_date in input_df.index:
        if index_date.date() in national_holidays:
            input_df.at[index_date, "Holidays_flag"] = 1

    return input_df




def TimeRelatedFeatureConstructor(df):
  """
  Works only in a dataframe as input: run the other functions first.
  Extracts time-related features
  """
  TimeFeaturesToExtract=["day_of_week","weekend","hour","minute",] #consider to add more
  dtfs=DatetimeFeatures(variables="index", features_to_extract=TimeFeaturesToExtract, drop_original=False)
  df=dtfs.fit_transform(df)

  CyclicalFeaturesToExtract=["day_of_week","hour","minute",]
  cyclical_dtfs=CyclicalFeatures(variables=CyclicalFeaturesToExtract,drop_original=False)
  df=cyclical_dtfs.fit_transform(df)
  return df


def FourierFeatureConstructor(df,granularity,fourier_terms_list):

    number_part = ''.join(filter(str.isdigit, granularity))
    number_int = int(number_part)
    minutes4hour=60/number_int

    Fourier_Transformer=FourierFeatures(
        sp_list=[minutes4hour, 24*minutes4hour,24*7*minutes4hour,24*30*minutes4hour],   # hourly, 24 is daily seasonality *12 because for 60 min we have 12 intervals,  and 24*7 is weekly seasonality
        fourier_terms_list=fourier_terms_list,
        freq=granularity, #not necessery
        keep_original_columns=True,

    )

    Fourier_Transformer.fit(df)
    df=Fourier_Transformer.transform(df)
    return df

def WindowFeaturesConstructor(df, granularity, ListWithNoFlags):
    """
    This is a function that makes a list of 4 window features starting from double the granularity and following by doubling the previous value
    """
    number_part = ''.join(filter(str.isdigit, granularity))
    number_int = int(number_part)
    double_granularity = 2 * number_int
    time_intervals = [double_granularity]

    # Calculate subsequent values
    for i in range(3):
        time_intervals.append(time_intervals[-1] * 2)

    windowlist = [interval // number_int for interval in time_intervals]  # Corrected division
    functionsList = ["mean", "std"]
    WindownFeatureTransformer = WindowFeatures(variables=ListWithNoFlags,
                                               functions=functionsList,
                                               window=windowlist,
                                               freq=granularity,
                                               drop_original=False)

    df = WindownFeatureTransformer.fit_transform(df)
    return df

def ExpandingWindowFeatureConstructor(df,ListWithNoFlags):
  functionsList=["mean","std"]
  frequency = pd.infer_freq(df.index) #infer the frequency from the dataframe
  ExpandingWindownFeatureTransformer=ExpandingWindowFeatures(variables=ListWithNoFlags,
                                                           functions=functionsList,
                                                           freq=frequency, #I put the freq to shift it down! but now it is performed automatically!
                                                           drop_original=False)
  df=ExpandingWindownFeatureTransformer.fit_transform(df)
  return df

def WeightedLinearFeatureMaker(df,ListWithNoFlags,granularity):
  """
  This is a function that takes the original DF and modifies the continious value columns
  Inputs: Dataframe, List of columns that are continous values, daily window to slide, weights of the values
  """
  number_part = ''.join(filter(str.isdigit, granularity))
  Minutedensity=int(number_part)
  Window=int((60/Minutedensity)*24) #288 means a daily window
  weights=np.arange(1,Window+1)

  # if i had hourly data then i would have had np.arange(1,24*7) for a weekly window

  def weighted_mean (x,weights):
    return (weights*x).sum()/weights.sum()

  def weighted_std(x,weights):
    mean_w= weighted_mean(x, weights)
    var_w= (weights* (x-mean_w)**2).sum()/weights.sum()
    return np.sqrt(var_w)

  # LETS make the weighted mean column
  for i in ListWithNoFlags:
    result=(
        df[i]
        .rolling(window=Window) #here we pick a window size. Needs to be the same as the len(weights)
        .apply(weighted_mean, args=(weights,))
        .shift(1)#shift by 1 to avoid data leakage
        .to_frame()#convert series to df
        )

    result.columns=[str(i)+"_weighted_"+str(Window)+"_mean"]
    df=df.join(result)

  for i in ListWithNoFlags:
    result=(
        df[i]
        .rolling(window=Window) #here we pick a window size. Needs to be the same as the len(weights)
        .apply(weighted_std, args=(weights,))
        .shift(1)#shift by 1 to avoid data leakage
        .to_frame()#convert series to df
        )

    result.columns=[str(i)+"_weighted_"+str(Window)+"_std"]
    df=df.join(result)
  return df

def ExpWeightMeanMaker(df,ListWithNoFlags,granularity):
  """
  This is a function that makes exp weighted average with a sliding window approach
  """
  number_part = ''.join(filter(str.isdigit, granularity))
  Minutedensity=int(number_part)
  Window=int((60/Minutedensity)*24) #288 means a daily window

  def exp_weights(alpha,window_size):
    """
    a function to calculate the weights for every single component of our sliding windown
    """
    weights=np.ones(window_size) #initializing weights
    for ix in range(window_size):
      weights[ix]=(1-alpha)**(window_size-1-ix)
    return weights

  def exp_weighted_mean(x):
    """
    a functions that calculates the exp weigted mean
    """

    weights=exp_weights(alpha=0.05, window_size=len(x)) # HERE WE SET THE ALPHA
    return (weights*x).sum()/weights.sum()

  for i in ListWithNoFlags:
    result=(
        df[i]
        .rolling(window=int(Window))
        .agg([exp_weighted_mean])
        .shift(1)
    )


    result.columns=[str(i)+"_Exp_weighted_"+str(Window)+"_SL.win"]
    df=df.join(result)
  return df

def WeightedExponentialExpandingWindow(df,ListWithNoFlags,alpha):
  """
  This is a funtion that takes as input the df,a list of continuous values and the alpha.
  Outputs: all continuous features on the df that are "mean" and "std"
  """

  for i in ListWithNoFlags:
    df[[str(i)+"_ewm_mean_expanding.win",str(i)+"ewm_std_expanding.win"]]= (
                                              df[i].ewm(alpha=alpha).
                                              agg(["mean","std"])
                                              .shift(1)
                                            )
  return df

def FeatureLagger(df,ListOfFeatures,granularity,PredictionHorizon):

    time_intervals = []
    number_part = ''.join(filter(str.isdigit, granularity))
    Minutedensity=int(number_part)
    end_in_day=int((PredictionHorizon)/(Minutedensity))
    for i in range(1, 1+end_in_day):  # 24 hours * 60 minutes / 15 minutes = 96 intervals
        time_intervals.append(f"{i * 15}min")

    lag_transformer= LagFeatures(variables=ListOfFeatures,
                                freq=time_intervals,
                                drop_original=False) #make a lagger transformer drop all original features

    df=lag_transformer.fit_transform(df) # transform the features to DF joined
    return df
def separate_future_past_features(df_columns):
    """
    Separates future and past features from a list of dataframe columns.

    Args:
        df_columns (list): A list of column names from the dataframe.

    Returns:
        dict: A dictionary with keys 'future_features' and 'past_features', containing the respective lists of column names.
    """
    future_keywords = ['sin', 'cos', 'weekend', 'hour', 'holiday', 'minute', 'day','+']
    
    future_features = []
    past_features = []
    
    for col in df_columns:
        # Check if the column contains "+" in its name to classify as a past feature
        if any(keyword in col.lower() for keyword in future_keywords):
            future_features.append(col)
        # Columns that don't meet the above conditions are considered past features by default
        else:
            past_features.append(col)

    return future_features, past_features

def plot_predictions(model_name, df_validation_y, all_preds_unscaled):
    plt.figure(figsize=(14, 7))
    
    # Plot actual values
    plt.plot(df_validation_y.index, df_validation_y.values, label='Actual Values', color='blue', linewidth=2)
    
    # Plot predicted values
    plt.plot(all_preds_unscaled.index, all_preds_unscaled.values, label=f'Predicted Values ({model_name})', color='red', linestyle='-', linewidth=2)
    
    # Add labels and title
    plt.xlabel('Time')
    plt.ylabel('Power')
    plt.title(f'Actual vs Predicted Values ({model_name})')
    
    # Add legend
    plt.legend()
    
    # Rotate x-ticks for better readability
    plt.xticks(rotation=45)
    
    # Show the plot
    plt.tight_layout()
    plt.show()


import seaborn as sns

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error


# Start

In [ ]:
# Load the DF:-
file = r"C:\Users\gkeep\OneDrive\Desktop\Thesis_coding\Building_Load_15min_Kw.csv"  # Replace with your file name
df = pd.read_csv(file,index_col=0, parse_dates=[0])
df=df[df.index.year.isin([2013])]
df=df[df.index.month.isin([1,2,3,4,5,6])]


# Step 1: Convert "Unnamed: 0" to a proper datetime format
#df.index= pd.to_datetime(df.index)

# Define parameters for processing
OriginalFeatures = ["Temp (C)", "Dew Point Temp (C)", "Rel Hum (%)", "Wind Dir (10s deg)" , "Stn Press (kPa)" , "Wind Spd (km/h)" , "Visibility (km)" , "WHE (kW)" ]
target = "WHE (kW)"
Minutedensity = 15
granularity = "15min"
PredictionHorizon=prediction_horizon = 1440 # in minutes
fourier_terms_list = [2, 2, 2, 2]

# Create Features:-
df = CreateWorkHourFeature(df)
df = HolidayFeatureCreator(df)
non_flag_columns, flag_columns = ListCreatorFlagger(df)
print(f"Non-flag columns: {non_flag_columns}")
print(f"Flag columns: {flag_columns}")
df = FeatureLagger(df, ListOfFeatures=non_flag_columns, granularity=granularity, PredictionHorizon=prediction_horizon)
# Add Fourier features
df = FourierFeatureConstructor(df, granularity=granularity, fourier_terms_list=fourier_terms_list)
# Add window features
df = WindowFeaturesConstructor(df, granularity=granularity, ListWithNoFlags=non_flag_columns)
# Add time-related features
df = TimeRelatedFeatureConstructor(df)


df[target+"_+lag"+str(PredictionHorizon)]=df[target].shift(int(-PredictionHorizon/Minutedensity))
target=target+"_+lag"+str(PredictionHorizon)
df.dropna(inplace=True)
has_nan = df.isnull().values.any()
print(f"Does the DataFrame have NaN values? {has_nan}")

In [ ]:
features_after_FS1_df = pd.read_csv(r'C:\Users\gkeep\OneDrive\Desktop\Thesis_coding\Selected_features_18.02.25_1-6.csv')
features_after_FS1 = features_after_FS1_df["Features"].tolist()
features_after_FS1.append(target)
features_after_FS1
df = df[features_after_FS1]

# Split

In [ ]:
# Step 1: Determine the number of rows in the DataFrame
total_size = len(df)

# Step 2: Define the number of rows for validation and test sets
validation_test_size = 96 * 7 * 2

# Step 3: Split the data
DF_training = df.iloc[:total_size - 2 * validation_test_size]
DF_validation = df.iloc[total_size - 2 * validation_test_size:total_size - validation_test_size]
DF_test = df.iloc[total_size - validation_test_size:]

# Step 4: Verify the sizes of each set
print("Training set size:", len(DF_training))
print("Validation set size:", len(DF_validation))
print("Test set size:", len(DF_test))

X_train, y_train = DF_training.drop(columns=['Demand MW_+lag1440']), DF_training['Demand MW_+lag1440']
x_validation, y_validation = DF_validation.drop(columns=['Demand MW_+lag1440']), DF_validation['Demand MW_+lag1440']
X_test, y_test = DF_test.drop(columns=['Demand MW_+lag1440']), DF_test['Demand MW_+lag1440']

# TCN

In [ ]:
df = df.iloc[-2688:]

df_1=pd.read_csv(r'C:\Users\gkeep\Desktop\Results\DNN\TCN_28.02_n=100_256\TCN_final_predictions.csv') # Validation Predictions
df_2=pd.read_csv(r'C:\Users\gkeep\Desktop\Results\DNN\TEST\TCN_07.03_Test_Results\TCN_final_predictions.csv')         # Test predictions

# Concatenate both DataFrames vertically
new_df = pd.concat([df_1, df_2], ignore_index=True)

df["Target"] = new_df["TCN_unscaled"].values  # Ensure values are aligned

df=df.drop(columns=['WHE (kW)_+lag1440'])

DF_test = df.iloc[-1344:]

## Normal Split

In [ ]:
import os
import json
import optuna
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

# ----------------------
# User-defined paths
# ----------------------
output_dir = r"C:\Users\gkeep\Desktop\Results\Regressors\TCN\ElasticNet"
os.makedirs(output_dir, exist_ok=True)

# ----------------------
# Load or define your data
# ----------------------
# Replace with your actual data
# df = pd.read_csv("your_data.csv", index_col=0, parse_dates=[0])
# For example:
# df = pd.read_csv("my_dataset.csv")
# DF_test = pd.read_csv("my_testset.csv")

# Placeholder (make sure to replace!)
# df, DF_test must contain "Target" column
#assert 'df' in globals() and 'DF_test' in globals(), "Load your dataset as df and DF_test first."

X_train = df.drop(columns=['Target'])
y_train = df['Target']
X_test = DF_test.drop(columns=['Target'])
y_test = DF_test['Target']

# ----------------------
# Scaling
# ----------------------
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ----------------------
# Optuna Optimization
# ----------------------
def objective(trial):
    params = {
        "alpha": trial.suggest_loguniform("alpha", 1e-4, 10.0),
        "l1_ratio": trial.suggest_uniform("l1_ratio", 0.0, 1.0),
        "fit_intercept": trial.suggest_categorical("fit_intercept", [True, False])
    }

    model = ElasticNet(**params, max_iter=10000, random_state=42)
    tscv = TimeSeriesSplit(n_splits=5)
    mse_scores = []

    for train_idx, val_idx in tscv.split(X_train_scaled):
        X_tr, X_val = X_train_scaled[train_idx], X_train_scaled[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

        model.fit(X_tr, y_tr)
        preds = model.predict(X_val)
        mse_scores.append(mean_squared_error(y_val, preds))

    return np.mean(mse_scores)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

best_params = study.best_params
print("Best Parameters:", best_params)

# ----------------------
# Train on full training data
# ----------------------
final_model = ElasticNet(**best_params, max_iter=10000, random_state=42)
final_model.fit(X_train_scaled, y_train)
y_pred = final_model.predict(X_test_scaled)
test_mse = mean_squared_error(y_test, y_pred)
print(f"Test MSE: {test_mse:.4f}")

# ----------------------
# Save predictions
# ----------------------
pred_df = pd.DataFrame({"Actual": y_test, "Predicted": y_pred}, index=y_test.index)
pred_df.to_csv(os.path.join(output_dir, "ElasticNet_predictions.csv"))

# ----------------------
# Save results
# ----------------------
results = {
    "Test MSE": test_mse,
    "Best Hyperparameters": best_params
}
with open(os.path.join(output_dir, "ElasticNet_results.json"), "w") as f:
    json.dump(results, f, indent=4)

# ----------------------
# Feature Importance Plot
# ----------------------
plt.figure(figsize=(12, 6))
plt.barh(X_train.columns, np.abs(final_model.coef_))
plt.xlabel("Coefficient Magnitude")
plt.ylabel("Feature")
plt.title("ElasticNet Feature Importance")
plt.tight_layout()
plt.show()

# ----------------------
# Plot Predictions
# ----------------------
plt.figure(figsize=(12, 6))
plt.plot(y_test.index, y_test, label="Actual", linewidth=1)
plt.plot(y_test.index, y_pred, label="ElasticNet Prediction", color="red", linewidth=1)
plt.xlabel("Time")
plt.ylabel("Target")
plt.title("ElasticNet Forecast vs Actual")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import optuna
import lightgbm as lgb
import xgboost as xgb
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit

# Define training and test sets
X_train = df.drop(columns=['Target']) # Train on Validation + Test
y_train = df['Target']

X_test = DF_test.drop(columns=['Target']) # Predict on Test
y_test = DF_test['Target']

# Define models and their hyperparameter search spaces
models = {
    "LightGBM": {
        "model": lgb.LGBMRegressor,
        "params": lambda trial: {
            "num_leaves": trial.suggest_int('num_leaves', 31, 100),
            "learning_rate": trial.suggest_loguniform('learning_rate', 0.01, 0.1),
            "n_estimators": trial.suggest_int('n_estimators', 100, 1000),
            "max_depth": trial.suggest_categorical('max_depth', [-1, 10, 20])
        }
    },
    "DecisionTree": {
        "model": DecisionTreeRegressor,
        "params": lambda trial: {
            "max_depth": trial.suggest_categorical('max_depth', [5, 10, 20, None]),
            "min_samples_split": trial.suggest_int('min_samples_split', 2, 10),
            "min_samples_leaf": trial.suggest_int('min_samples_leaf', 1, 4),
            "max_features": trial.suggest_categorical('max_features', [None, 'sqrt', 'log2'])
        }
    },
    "XGBoost": {
        "model": xgb.XGBRegressor,
        "params": lambda trial: {
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "n_estimators": trial.suggest_int("n_estimators", 100, 2000, step=100),
            "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.2),
            "subsample": trial.suggest_categorical("subsample", [0.7, 0.8, 1.0]),
            "colsample_bytree": trial.suggest_categorical("colsample_bytree", [0.7, 0.8, 1.0]),
        }
    },
    "RandomForest": {
        "model": RandomForestRegressor,
        "params": lambda trial: {
            "max_depth": trial.suggest_categorical('max_depth', [10, 20, 30]),
            "n_estimators": trial.suggest_int("n_estimators", 100, 500, step=100),
            "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 4),
            "bootstrap": trial.suggest_categorical("bootstrap", [True, False])
        }
    }
}

# TimeSeries Cross-validation strategy
tscv = TimeSeriesSplit(n_splits=5)

# Store results
results = {}

# Loop through models
for model_name, model_info in models.items():
    
    def objective(trial):
        params = model_info["params"](trial)
        model = model_info["model"](**params, random_state=42)

        mse_scores = []
        
        # Perform manual cross-validation using TimeSeriesSplit
        for train_index, val_index in tscv.split(X_train):
            X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
            y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

            model.fit(X_train_fold, y_train_fold)
            y_pred_fold = model.predict(X_val_fold)
            mse_scores.append(mean_squared_error(y_val_fold, y_pred_fold))

        return sum(mse_scores) / len(mse_scores)  # Return average MSE
    
    # Optimize hyperparameters using Optuna
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=50)

    best_params = study.best_params
    print(f"\nBest Hyperparameters for {model_name}: {best_params}\n")

    # Train the best model on the full training set
    best_model = model_info["model"](**best_params, random_state=42)
    best_model.fit(X_train, y_train)

    # Predict on test set
    y_pred_test = best_model.predict(X_test)
    mse_test = mean_squared_error(y_test, y_pred_test)
    results[model_name] = mse_test

    print(f"{model_name} - Mean Squared Error (Test): {mse_test:.4f}")

    # Feature Importance Analysis
    if hasattr(best_model, "feature_importances_"):
        plt.figure(figsize=(12, 6))
        plt.barh(X_train.columns, best_model.feature_importances_, align='center')
        plt.xlabel("Feature Importance")
        plt.ylabel("Feature")
        plt.title(f"{model_name} Feature Importance Analysis")
        plt.show()

    # Plot Actual vs. Predicted Values
    plt.figure(figsize=(12, 6))
    plt.plot(y_test.index, y_test, label="Actual Demand MW_+lag1440")
    plt.plot(y_test.index, y_pred_test, label=f"Predicted Demand MW_+lag1440 ({model_name})", color='red')
    plt.legend()
    plt.xlabel("Time")
    plt.ylabel("Demand MW_+lag1440")
    plt.title(f"{model_name} Time-Series Forecasting")
    plt.show()

# Print final results
print("\nFinal Model Performance:")
for model, mse in results.items():
    print(f"{model}: Test MSE = {mse:.4f}")


In [ ]:
import optuna
import lightgbm as lgb
import xgboost as xgb
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit

# Create output directory if not exists
output_dir = r"C:\Users\gkeep\Desktop\Results\Regressors\TCN"
os.makedirs(output_dir, exist_ok=True)

# Define training and test sets
X_train = df.drop(columns=['Target'])  # Train on Validation + Test
y_train = df['Target']

X_test = DF_test.drop(columns=['Target'])  # Predict on Test
y_test = DF_test['Target']

# Define models and their hyperparameter search spaces
models = {
    "LightGBM": {
        "model": lgb.LGBMRegressor,
        "params": lambda trial: {
            "num_leaves": trial.suggest_int('num_leaves', 31, 100),
            "learning_rate": trial.suggest_loguniform('learning_rate', 0.01, 0.1),
            "n_estimators": trial.suggest_int('n_estimators', 100, 1000),
            "max_depth": trial.suggest_categorical('max_depth', [-1, 10, 20])
        }
    },
    "DecisionTree": {
        "model": DecisionTreeRegressor,
        "params": lambda trial: {
            "max_depth": trial.suggest_categorical('max_depth', [5, 10, 20, None]),
            "min_samples_split": trial.suggest_int('min_samples_split', 2, 10),
            "min_samples_leaf": trial.suggest_int('min_samples_leaf', 1, 4),
            "max_features": trial.suggest_categorical('max_features', [None, 'sqrt', 'log2'])
        }
    },
    "XGBoost": {
        "model": xgb.XGBRegressor,
        "params": lambda trial: {
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "n_estimators": trial.suggest_int("n_estimators", 100, 2000, step=100),
            "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.2),
            "subsample": trial.suggest_categorical("subsample", [0.7, 0.8, 1.0]),
            "colsample_bytree": trial.suggest_categorical("colsample_bytree", [0.7, 0.8, 1.0]),
        }
    },
    "RandomForest": {
        "model": RandomForestRegressor,
        "params": lambda trial: {
            "max_depth": trial.suggest_categorical('max_depth', [10, 20, 30]),
            "n_estimators": trial.suggest_int("n_estimators", 100, 500, step=100),
            "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 4),
            "bootstrap": trial.suggest_categorical("bootstrap", [True, False])
        }
    }
}

# TimeSeries Cross-validation strategy
tscv = TimeSeriesSplit(n_splits=5)

# Store results
results = {}

# Loop through models
for model_name, model_info in models.items():
    
    def objective(trial):
        params = model_info["params"](trial)
        model = model_info["model"](**params, random_state=42)

        mse_scores = []
        
        # Perform manual cross-validation using TimeSeriesSplit
        for train_index, val_index in tscv.split(X_train):
            X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
            y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

            model.fit(X_train_fold, y_train_fold)
            y_pred_fold = model.predict(X_val_fold)
            mse_scores.append(mean_squared_error(y_val_fold, y_pred_fold))

        return sum(mse_scores) / len(mse_scores)  # Return average MSE
    
    # Optimize hyperparameters using Optuna
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=50)

    best_params = study.best_params
    print(f"\nBest Hyperparameters for {model_name}: {best_params}\n")

    # Train the best model on the full training set
    best_model = model_info["model"](**best_params, random_state=42)
    best_model.fit(X_train, y_train)

    # Predict on test set
    y_pred_test = best_model.predict(X_test)
    mse_test = mean_squared_error(y_test, y_pred_test)

    # Save predictions to CSV
    pred_df = pd.DataFrame({"Actual": y_test, "Predicted": y_pred_test}, index=y_test.index)
    pred_csv_path = os.path.join(output_dir, f"{model_name}_predictions.csv")
    pred_df.to_csv(pred_csv_path, index=True)
    
    # Save results
    results[model_name] = {
        "Test MSE": mse_test,
        "Best Hyperparameters": best_params
    }

    print(f"{model_name} - Mean Squared Error (Test): {mse_test:.4f}")

    # Feature Importance Analysis
    if hasattr(best_model, "feature_importances_"):
        plt.figure(figsize=(12, 6))
        plt.barh(X_train.columns, best_model.feature_importances_, align='center')
        plt.xlabel("Feature Importance")
        plt.ylabel("Feature")
        plt.title(f"{model_name} Feature Importance Analysis")
        plt.show()

    # Plot Actual vs. Predicted Values
    plt.figure(figsize=(12, 6))
    plt.plot(y_test.index, y_test, label="Actual Predictions" , linewidth=1)
    plt.plot(y_test.index, y_pred_test, label=f"Predicted with ({model_name})", color='red' , linestyle='-' , linewidth=1)
    plt.legend()
    plt.xlabel("Time")
    plt.ylabel("Predictions")
    plt.title(f"{model_name} Time-Series Forecasting")
    plt.show()

# Save all results to JSON file
results_json_path = os.path.join(output_dir, "summary_results.json")
with open(results_json_path, "w") as f:
    json.dump(results, f, indent=4)

# Print final results
print("\nFinal Model Performance:")
for model, data in results.items():
    print(f"{model}: Test MSE = {data['Test MSE']:.4f}")


In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Define training and test sets
X_train = df.drop(columns=['Target']) # Train on Validation + Test
y_train = df['Target']

X_test = DF_test.drop(columns=['Target']) # Predict on Test
y_test = DF_test['Target']

# 🚨 **Ensure NO Data Leakage: Fit scaler ONLY on training set**
scaler = MinMaxScaler()
scaler_target = MinMaxScaler()

# Fit the feature scaler only on training set
scaler.fit(X_train)  # Fit on training features only

# Scale features for both training and test sets using the fitted scaler
X_train_scaled = X_train.copy()
X_train_scaled = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns)

X_test_scaled = X_test.copy()
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

# 🚨 **Fit the target scaler ONLY on the original (unscaled) training target values**
scaler_target.fit(y_train.values.reshape(-1, 1))

# Transform target variable separately for training and test sets
y_train_scaled = pd.DataFrame(scaler_target.transform(y_train.values.reshape(-1, 1)), columns=['Target'])
y_test_scaled = pd.DataFrame(scaler_target.transform(y_test.values.reshape(-1, 1)), columns=['Target'])

# 🔹 Print shapes to verify everything is correct
print(f"X_train: {X_train.shape}, X_train_scaled: {X_train_scaled.shape}")
print(f"X_test: {X_test.shape}, X_test_scaled: {X_test_scaled.shape}")

# 🔎 **Final Sanity Check**
print("Training Target Min/Max (Scaled):", y_train_scaled.min().values[0], y_train_scaled.max().values[0])
print("Test Target Min/Max (Scaled):", y_test_scaled.min().values[0], y_test_scaled.max().values[0])



In [ ]:
import optuna
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler


# Define Optuna objective function for Lasso hyperparameter tuning
def objective(trial):
    alpha = trial.suggest_float("alpha", 1e-4, 10.0, log=True)  # Search for best alpha (log scale)
    
    # Train Lasso model on TRAINING SET ONLY
    model = Lasso(alpha=alpha, random_state=42)
    model.fit(X_train_scaled, y_train_scaled)

    # Predict on training set (for Optuna optimization)
    y_pred_train_scaled = model.predict(X_train_scaled)
    y_pred_train = scaler_target.inverse_transform(y_pred_train_scaled.reshape(-1, 1)).flatten()

    # Calculate Mean Squared Error (MSE) on training set
    mse_train = mean_squared_error(y_train, y_pred_train)
    
    return mse_train  # Optuna will minimize training MSE

# Run Optuna study to find the best hyperparameters
study = optuna.create_study(direction="minimize")  # Minimize training MSE
study.optimize(objective, n_trials=50, n_jobs=-1)  # Run 50 trials in parallel

# Best hyperparameters from Optuna
best_alpha = study.best_params["alpha"]
print("Best Hyperparameters (Lasso):", study.best_params)

# Train the final Lasso model using the best alpha on FULL TRAINING DATA (X_train_scaled)
best_lasso_model = Lasso(alpha=best_alpha, random_state=42)
best_lasso_model.fit(X_train_scaled, y_train_scaled)

# Predict on test set using the final trained model
y_pred_test_scaled = best_lasso_model.predict(X_test_scaled)
y_pred_test = scaler_target.inverse_transform(y_pred_test_scaled.reshape(-1, 1)).flatten()

# Calculate MSE on test set
mse_test = mean_squared_error(y_test, y_pred_test)
print(f"Mean Squared Error (Test): {mse_test:.4f}")

# 🔹 Feature Importance Analysis (Lasso Coefficients)
feature_importances = np.abs(best_lasso_model.coef_)
plt.figure(figsize=(12, 6))
plt.barh(X_train_scaled.columns, feature_importances, align='center')
plt.xlabel("Feature Importance")
plt.ylabel("Feature")
plt.title("Optimized Lasso Feature Importance Analysis (Optuna)")
plt.show()

# 🔹 Plot Actual vs. Predicted Values
plt.figure(figsize=(12, 6))
plt.plot(y_test.index, y_test, label="Actual Demand MW_+lag1440")
plt.plot(y_test.index, y_pred_test, label="Predicted Demand MW_+lag1440 (Lasso)", color='red')
plt.legend()
plt.xlabel("Time")
plt.ylabel("Demand MW_+lag1440")
plt.title("Lasso Time-Series Forecasting with Optuna")
plt.show()


# TSmixersx

In [ ]:
df = df.iloc[-2688:]

df_1=pd.read_csv(r'C:\Users\gkeep\Desktop\Results\DNN\TSMixerx_01.03_n=100_256\TSMixerx_final_predictions.csv') # Validation Predictions
df_2=pd.read_csv(r'C:\Users\gkeep\Desktop\Results\DNN\TEST\TSMixerx_11.03_Test_Results\TSMixerx_final_predictions.csv')         # Test predictions

# Concatenate both DataFrames vertically
new_df = pd.concat([df_1, df_2], ignore_index=True)

df["Target"] = new_df["TSMixerx_unscaled"].values  # Ensure values are aligned

df=df.drop(columns=['WHE (kW)_+lag1440'])

DF_test = df.iloc[-1344:]

## Normal Split

In [ ]:
import os
import json
import optuna
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

# ----------------------
# User-defined paths
# ----------------------
output_dir = r"C:\Users\gkeep\Desktop\Results\Regressors\TSMixerx\ElasticNet"
os.makedirs(output_dir, exist_ok=True)

# ----------------------
# Load or define your data
# ----------------------
# Replace with your actual data
# df = pd.read_csv("your_data.csv", index_col=0, parse_dates=[0])
# For example:
# df = pd.read_csv("my_dataset.csv")
# DF_test = pd.read_csv("my_testset.csv")

# Placeholder (make sure to replace!)
# df, DF_test must contain "Target" column
#assert 'df' in globals() and 'DF_test' in globals(), "Load your dataset as df and DF_test first."

X_train = df.drop(columns=['Target'])
y_train = df['Target']
X_test = DF_test.drop(columns=['Target'])
y_test = DF_test['Target']

# ----------------------
# Scaling
# ----------------------
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ----------------------
# Optuna Optimization
# ----------------------
def objective(trial):
    params = {
        "alpha": trial.suggest_loguniform("alpha", 1e-4, 10.0),
        "l1_ratio": trial.suggest_uniform("l1_ratio", 0.0, 1.0),
        "fit_intercept": trial.suggest_categorical("fit_intercept", [True, False])
    }

    model = ElasticNet(**params, max_iter=10000, random_state=42)
    tscv = TimeSeriesSplit(n_splits=5)
    mse_scores = []

    for train_idx, val_idx in tscv.split(X_train_scaled):
        X_tr, X_val = X_train_scaled[train_idx], X_train_scaled[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

        model.fit(X_tr, y_tr)
        preds = model.predict(X_val)
        mse_scores.append(mean_squared_error(y_val, preds))

    return np.mean(mse_scores)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

best_params = study.best_params
print("Best Parameters:", best_params)

# ----------------------
# Train on full training data
# ----------------------
final_model = ElasticNet(**best_params, max_iter=10000, random_state=42)
final_model.fit(X_train_scaled, y_train)
y_pred = final_model.predict(X_test_scaled)
test_mse = mean_squared_error(y_test, y_pred)
print(f"Test MSE: {test_mse:.4f}")

# ----------------------
# Save predictions
# ----------------------
pred_df = pd.DataFrame({"Actual": y_test, "Predicted": y_pred}, index=y_test.index)
pred_df.to_csv(os.path.join(output_dir, "ElasticNet_predictions.csv"))

# ----------------------
# Save results
# ----------------------
results = {
    "Test MSE": test_mse,
    "Best Hyperparameters": best_params
}
with open(os.path.join(output_dir, "ElasticNet_results.json"), "w") as f:
    json.dump(results, f, indent=4)

# ----------------------
# Feature Importance Plot
# ----------------------
plt.figure(figsize=(12, 6))
plt.barh(X_train.columns, np.abs(final_model.coef_))
plt.xlabel("Coefficient Magnitude")
plt.ylabel("Feature")
plt.title("ElasticNet Feature Importance")
plt.tight_layout()
plt.show()

# ----------------------
# Plot Predictions
# ----------------------
plt.figure(figsize=(12, 6))
plt.plot(y_test.index, y_test, label="Actual", linewidth=1)
plt.plot(y_test.index, y_pred, label="ElasticNet Prediction", color="red", linewidth=1)
plt.xlabel("Time")
plt.ylabel("Target")
plt.title("ElasticNet Forecast vs Actual")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import optuna
import lightgbm as lgb
import xgboost as xgb
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit

# Create output directory if not exists
output_dir = r"C:\Users\gkeep\Desktop\Results\Regressors\TSMixerx"
os.makedirs(output_dir, exist_ok=True)

# Define training and test sets
X_train = df.drop(columns=['Target'])  # Train on Validation + Test
y_train = df['Target']

X_test = DF_test.drop(columns=['Target'])  # Predict on Test
y_test = DF_test['Target']

# Define models and their hyperparameter search spaces
models = {
    "LightGBM": {
        "model": lgb.LGBMRegressor,
        "params": lambda trial: {
            "num_leaves": trial.suggest_int('num_leaves', 31, 100),
            "learning_rate": trial.suggest_loguniform('learning_rate', 0.01, 0.1),
            "n_estimators": trial.suggest_int('n_estimators', 100, 1000),
            "max_depth": trial.suggest_categorical('max_depth', [-1, 10, 20])
        }
    },
    "DecisionTree": {
        "model": DecisionTreeRegressor,
        "params": lambda trial: {
            "max_depth": trial.suggest_categorical('max_depth', [5, 10, 20, None]),
            "min_samples_split": trial.suggest_int('min_samples_split', 2, 10),
            "min_samples_leaf": trial.suggest_int('min_samples_leaf', 1, 4),
            "max_features": trial.suggest_categorical('max_features', [None, 'sqrt', 'log2'])
        }
    },
    "XGBoost": {
        "model": xgb.XGBRegressor,
        "params": lambda trial: {
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "n_estimators": trial.suggest_int("n_estimators", 100, 2000, step=100),
            "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.2),
            "subsample": trial.suggest_categorical("subsample", [0.7, 0.8, 1.0]),
            "colsample_bytree": trial.suggest_categorical("colsample_bytree", [0.7, 0.8, 1.0]),
        }
    },
    "RandomForest": {
        "model": RandomForestRegressor,
        "params": lambda trial: {
            "max_depth": trial.suggest_categorical('max_depth', [10, 20, 30]),
            "n_estimators": trial.suggest_int("n_estimators", 100, 500, step=100),
            "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 4),
            "bootstrap": trial.suggest_categorical("bootstrap", [True, False])
        }
    }
}

# TimeSeries Cross-validation strategy
tscv = TimeSeriesSplit(n_splits=5)

# Store results
results = {}

# Loop through models
for model_name, model_info in models.items():
    
    def objective(trial):
        params = model_info["params"](trial)
        model = model_info["model"](**params, random_state=42)

        mse_scores = []
        
        # Perform manual cross-validation using TimeSeriesSplit
        for train_index, val_index in tscv.split(X_train):
            X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
            y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

            model.fit(X_train_fold, y_train_fold)
            y_pred_fold = model.predict(X_val_fold)
            mse_scores.append(mean_squared_error(y_val_fold, y_pred_fold))

        return sum(mse_scores) / len(mse_scores)  # Return average MSE
    
    # Optimize hyperparameters using Optuna
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=50)

    best_params = study.best_params
    print(f"\nBest Hyperparameters for {model_name}: {best_params}\n")

    # Train the best model on the full training set
    best_model = model_info["model"](**best_params, random_state=42)
    best_model.fit(X_train, y_train)

    # Predict on test set
    y_pred_test = best_model.predict(X_test)
    mse_test = mean_squared_error(y_test, y_pred_test)

    # Save predictions to CSV
    pred_df = pd.DataFrame({"Actual": y_test, "Predicted": y_pred_test}, index=y_test.index)
    pred_csv_path = os.path.join(output_dir, f"{model_name}_predictions.csv")
    pred_df.to_csv(pred_csv_path, index=True)
    
    # Save results
    results[model_name] = {
        "Test MSE": mse_test,
        "Best Hyperparameters": best_params
    }

    print(f"{model_name} - Mean Squared Error (Test): {mse_test:.4f}")

    # Feature Importance Analysis
    if hasattr(best_model, "feature_importances_"):
        plt.figure(figsize=(12, 6))
        plt.barh(X_train.columns, best_model.feature_importances_, align='center')
        plt.xlabel("Feature Importance")
        plt.ylabel("Feature")
        plt.title(f"{model_name} Feature Importance Analysis")
        plt.show()

    # Plot Actual vs. Predicted Values
    plt.figure(figsize=(12, 6))
    plt.plot(y_test.index, y_test, label="Actual Predictions" , linewidth=1)
    plt.plot(y_test.index, y_pred_test, label=f"Predicted with ({model_name})", color='red' , linestyle='-' , linewidth=1)
    plt.legend()
    plt.xlabel("Time")
    plt.ylabel("Predictions")
    plt.title(f"{model_name} Time-Series Forecasting")
    plt.show()

# Save all results to JSON file
results_json_path = os.path.join(output_dir, "summary_results.json")
with open(results_json_path, "w") as f:
    json.dump(results, f, indent=4)

# Print final results
print("\nFinal Model Performance:")
for model, data in results.items():
    print(f"{model}: Test MSE = {data['Test MSE']:.4f}")


# NBEATSx

In [ ]:
df = df.iloc[-2688:]

df_1=pd.read_csv(r'C:\Users\gkeep\Desktop\Results\DNN\Trial\NBEATSx_15.05_n=100\NBEATSx_final_predictions.csv') # Validation Predictions
df_2=pd.read_csv(r'C:\Users\gkeep\Desktop\Results\DNN\TEST\NBEATSx_19.05_Test_Results\NBEATSx_final_predictions.csv')         # Test predictions

# Concatenate both DataFrames vertically
new_df = pd.concat([df_1, df_2], ignore_index=True)

df["Target"] = new_df["NBEATSx_unscaled"].values  # Ensure values are aligned

df=df.drop(columns=['WHE (kW)_+lag1440'])

DF_test = df.iloc[-1344:]

## Normal Split

In [ ]:
import os
import json
import optuna
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

# ----------------------
# User-defined paths
# ----------------------
output_dir = r"C:\Users\gkeep\Desktop\Results\Regressors\NBEATSx\ElasticNet"
os.makedirs(output_dir, exist_ok=True)

# ----------------------
# Load or define your data
# ----------------------
# Replace with your actual data
# df = pd.read_csv("your_data.csv", index_col=0, parse_dates=[0])
# For example:
# df = pd.read_csv("my_dataset.csv")
# DF_test = pd.read_csv("my_testset.csv")

# Placeholder (make sure to replace!)
# df, DF_test must contain "Target" column
#assert 'df' in globals() and 'DF_test' in globals(), "Load your dataset as df and DF_test first."

X_train = df.drop(columns=['Target'])
y_train = df['Target']
X_test = DF_test.drop(columns=['Target'])
y_test = DF_test['Target']

# ----------------------
# Scaling
# ----------------------
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ----------------------
# Optuna Optimization
# ----------------------
def objective(trial):
    params = {
        "alpha": trial.suggest_loguniform("alpha", 1e-4, 10.0),
        "l1_ratio": trial.suggest_uniform("l1_ratio", 0.0, 1.0),
        "fit_intercept": trial.suggest_categorical("fit_intercept", [True, False])
    }

    model = ElasticNet(**params, max_iter=10000, random_state=42)
    tscv = TimeSeriesSplit(n_splits=5)
    mse_scores = []

    for train_idx, val_idx in tscv.split(X_train_scaled):
        X_tr, X_val = X_train_scaled[train_idx], X_train_scaled[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

        model.fit(X_tr, y_tr)
        preds = model.predict(X_val)
        mse_scores.append(mean_squared_error(y_val, preds))

    return np.mean(mse_scores)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

best_params = study.best_params
print("Best Parameters:", best_params)

# ----------------------
# Train on full training data
# ----------------------
final_model = ElasticNet(**best_params, max_iter=10000, random_state=42)
final_model.fit(X_train_scaled, y_train)
y_pred = final_model.predict(X_test_scaled)
test_mse = mean_squared_error(y_test, y_pred)
print(f"Test MSE: {test_mse:.4f}")

# ----------------------
# Save predictions
# ----------------------
pred_df = pd.DataFrame({"Actual": y_test, "Predicted": y_pred}, index=y_test.index)
pred_df.to_csv(os.path.join(output_dir, "ElasticNet_predictions.csv"))

# ----------------------
# Save results
# ----------------------
results = {
    "Test MSE": test_mse,
    "Best Hyperparameters": best_params
}
with open(os.path.join(output_dir, "ElasticNet_results.json"), "w") as f:
    json.dump(results, f, indent=4)

# ----------------------
# Feature Importance Plot
# ----------------------
plt.figure(figsize=(12, 6))
plt.barh(X_train.columns, np.abs(final_model.coef_))
plt.xlabel("Coefficient Magnitude")
plt.ylabel("Feature")
plt.title("ElasticNet Feature Importance")
plt.tight_layout()
plt.show()

# ----------------------
# Plot Predictions
# ----------------------
plt.figure(figsize=(12, 6))
plt.plot(y_test.index, y_test, label="Actual", linewidth=1)
plt.plot(y_test.index, y_pred, label="ElasticNet Prediction", color="red", linewidth=1)
plt.xlabel("Time")
plt.ylabel("Target")
plt.title("ElasticNet Forecast vs Actual")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import optuna
import lightgbm as lgb
import xgboost as xgb
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit

# Create output directory if not exists
output_dir = r"C:\Users\gkeep\Desktop\Results\Regressors\NBEATSx"
os.makedirs(output_dir, exist_ok=True)

# Define training and test sets
X_train = df.drop(columns=['Target'])  # Train on Validation + Test
y_train = df['Target']

X_test = DF_test.drop(columns=['Target'])  # Predict on Test
y_test = DF_test['Target']

# Define models and their hyperparameter search spaces
models = {
    "LightGBM": {
        "model": lgb.LGBMRegressor,
        "params": lambda trial: {
            "num_leaves": trial.suggest_int('num_leaves', 31, 100),
            "learning_rate": trial.suggest_loguniform('learning_rate', 0.01, 0.1),
            "n_estimators": trial.suggest_int('n_estimators', 100, 1000),
            "max_depth": trial.suggest_categorical('max_depth', [-1, 10, 20])
        }
    },
    "DecisionTree": {
        "model": DecisionTreeRegressor,
        "params": lambda trial: {
            "max_depth": trial.suggest_categorical('max_depth', [5, 10, 20, None]),
            "min_samples_split": trial.suggest_int('min_samples_split', 2, 10),
            "min_samples_leaf": trial.suggest_int('min_samples_leaf', 1, 4),
            "max_features": trial.suggest_categorical('max_features', [None, 'sqrt', 'log2'])
        }
    },
    "XGBoost": {
        "model": xgb.XGBRegressor,
        "params": lambda trial: {
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "n_estimators": trial.suggest_int("n_estimators", 100, 2000, step=100),
            "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.2),
            "subsample": trial.suggest_categorical("subsample", [0.7, 0.8, 1.0]),
            "colsample_bytree": trial.suggest_categorical("colsample_bytree", [0.7, 0.8, 1.0]),
        }
    },
    "RandomForest": {
        "model": RandomForestRegressor,
        "params": lambda trial: {
            "max_depth": trial.suggest_categorical('max_depth', [10, 20, 30]),
            "n_estimators": trial.suggest_int("n_estimators", 100, 500, step=100),
            "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 4),
            "bootstrap": trial.suggest_categorical("bootstrap", [True, False])
        }
    }
}

# TimeSeries Cross-validation strategy
tscv = TimeSeriesSplit(n_splits=5)

# Store results
results = {}

# Loop through models
for model_name, model_info in models.items():
    
    def objective(trial):
        params = model_info["params"](trial)
        model = model_info["model"](**params, random_state=42)

        mse_scores = []
        
        # Perform manual cross-validation using TimeSeriesSplit
        for train_index, val_index in tscv.split(X_train):
            X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
            y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

            model.fit(X_train_fold, y_train_fold)
            y_pred_fold = model.predict(X_val_fold)
            mse_scores.append(mean_squared_error(y_val_fold, y_pred_fold))

        return sum(mse_scores) / len(mse_scores)  # Return average MSE
    
    # Optimize hyperparameters using Optuna
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=50)

    best_params = study.best_params
    print(f"\nBest Hyperparameters for {model_name}: {best_params}\n")

    # Train the best model on the full training set
    best_model = model_info["model"](**best_params, random_state=42)
    best_model.fit(X_train, y_train)

    # Predict on test set
    y_pred_test = best_model.predict(X_test)
    mse_test = mean_squared_error(y_test, y_pred_test)

    # Save predictions to CSV
    pred_df = pd.DataFrame({"Actual": y_test, "Predicted": y_pred_test}, index=y_test.index)
    pred_csv_path = os.path.join(output_dir, f"{model_name}_predictions.csv")
    pred_df.to_csv(pred_csv_path, index=True)
    
    # Save results
    results[model_name] = {
        "Test MSE": mse_test,
        "Best Hyperparameters": best_params
    }

    print(f"{model_name} - Mean Squared Error (Test): {mse_test:.4f}")

    # Feature Importance Analysis
    if hasattr(best_model, "feature_importances_"):
        plt.figure(figsize=(12, 6))
        plt.barh(X_train.columns, best_model.feature_importances_, align='center')
        plt.xlabel("Feature Importance")
        plt.ylabel("Feature")
        plt.title(f"{model_name} Feature Importance Analysis")
        plt.show()

    # Plot Actual vs. Predicted Values
    plt.figure(figsize=(12, 6))
    plt.plot(y_test.index, y_test, label="Actual Predictions" , linewidth=1)
    plt.plot(y_test.index, y_pred_test, label=f"Predicted with ({model_name})", color='red' , linestyle='-' , linewidth=1)
    plt.legend()
    plt.xlabel("Time")
    plt.ylabel("Predictions")
    plt.title(f"{model_name} Time-Series Forecasting")
    plt.show()

# Save all results to JSON file
results_json_path = os.path.join(output_dir, "summary_results.json")
with open(results_json_path, "w") as f:
    json.dump(results, f, indent=4)

# Print final results
print("\nFinal Model Performance:")
for model, data in results.items():
    print(f"{model}: Test MSE = {data['Test MSE']:.4f}")


# LSTM

In [ ]:
df = df.iloc[-2688:]

df_1=pd.read_csv(r'C:\Users\gkeep\Desktop\Results\DNN\LSTM_13.03_n=100\LSTM_final_predictions.csv') # Validation Predictions
df_2=pd.read_csv(r'C:\Users\gkeep\Desktop\Results\DNN\TEST\LSTM_14.03_Test_Results\LSTM_final_predictions.csv')         # Test predictions

# Concatenate both DataFrames vertically
new_df = pd.concat([df_1, df_2], ignore_index=True)

df["Target"] = new_df["LSTM_unscaled"].values  # Ensure values are aligned

df=df.drop(columns=['WHE (kW)_+lag1440'])

DF_test = df.iloc[-1344:]

## Normal Split

In [ ]:
import os
import json
import optuna
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

# ----------------------
# User-defined paths
# ----------------------
output_dir = r"C:\Users\gkeep\Desktop\Results\Regressors\LSTM\ElasticNet"
os.makedirs(output_dir, exist_ok=True)

# ----------------------
# Load or define your data
# ----------------------
# Replace with your actual data
# df = pd.read_csv("your_data.csv", index_col=0, parse_dates=[0])
# For example:
# df = pd.read_csv("my_dataset.csv")
# DF_test = pd.read_csv("my_testset.csv")

# Placeholder (make sure to replace!)
# df, DF_test must contain "Target" column
#assert 'df' in globals() and 'DF_test' in globals(), "Load your dataset as df and DF_test first."

X_train = df.drop(columns=['Target'])
y_train = df['Target']
X_test = DF_test.drop(columns=['Target'])
y_test = DF_test['Target']

# ----------------------
# Scaling
# ----------------------
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ----------------------
# Optuna Optimization
# ----------------------
def objective(trial):
    params = {
        "alpha": trial.suggest_loguniform("alpha", 1e-4, 10.0),
        "l1_ratio": trial.suggest_uniform("l1_ratio", 0.0, 1.0),
        "fit_intercept": trial.suggest_categorical("fit_intercept", [True, False])
    }

    model = ElasticNet(**params, max_iter=10000, random_state=42)
    tscv = TimeSeriesSplit(n_splits=5)
    mse_scores = []

    for train_idx, val_idx in tscv.split(X_train_scaled):
        X_tr, X_val = X_train_scaled[train_idx], X_train_scaled[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

        model.fit(X_tr, y_tr)
        preds = model.predict(X_val)
        mse_scores.append(mean_squared_error(y_val, preds))

    return np.mean(mse_scores)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

best_params = study.best_params
print("Best Parameters:", best_params)

# ----------------------
# Train on full training data
# ----------------------
final_model = ElasticNet(**best_params, max_iter=10000, random_state=42)
final_model.fit(X_train_scaled, y_train)
y_pred = final_model.predict(X_test_scaled)
test_mse = mean_squared_error(y_test, y_pred)
print(f"Test MSE: {test_mse:.4f}")

# ----------------------
# Save predictions
# ----------------------
pred_df = pd.DataFrame({"Actual": y_test, "Predicted": y_pred}, index=y_test.index)
pred_df.to_csv(os.path.join(output_dir, "ElasticNet_predictions.csv"))

# ----------------------
# Save results
# ----------------------
results = {
    "Test MSE": test_mse,
    "Best Hyperparameters": best_params
}
with open(os.path.join(output_dir, "ElasticNet_results.json"), "w") as f:
    json.dump(results, f, indent=4)

# ----------------------
# Feature Importance Plot
# ----------------------
plt.figure(figsize=(12, 6))
plt.barh(X_train.columns, np.abs(final_model.coef_))
plt.xlabel("Coefficient Magnitude")
plt.ylabel("Feature")
plt.title("ElasticNet Feature Importance")
plt.tight_layout()
plt.show()

# ----------------------
# Plot Predictions
# ----------------------
plt.figure(figsize=(12, 6))
plt.plot(y_test.index, y_test, label="Actual", linewidth=1)
plt.plot(y_test.index, y_pred, label="ElasticNet Prediction", color="red", linewidth=1)
plt.xlabel("Time")
plt.ylabel("Target")
plt.title("ElasticNet Forecast vs Actual")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import optuna
import lightgbm as lgb
import xgboost as xgb
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit

# Create output directory if not exists
output_dir = r"C:\Users\gkeep\Desktop\Results\Regressors\LSTM"
os.makedirs(output_dir, exist_ok=True)

# Define training and test sets
X_train = df.drop(columns=['Target'])  # Train on Validation + Test
y_train = df['Target']

X_test = DF_test.drop(columns=['Target'])  # Predict on Test
y_test = DF_test['Target']

# Define models and their hyperparameter search spaces
models = {
    "LightGBM": {
        "model": lgb.LGBMRegressor,
        "params": lambda trial: {
            "num_leaves": trial.suggest_int('num_leaves', 31, 100),
            "learning_rate": trial.suggest_loguniform('learning_rate', 0.01, 0.1),
            "n_estimators": trial.suggest_int('n_estimators', 100, 1000),
            "max_depth": trial.suggest_categorical('max_depth', [-1, 10, 20])
        }
    },
    "DecisionTree": {
        "model": DecisionTreeRegressor,
        "params": lambda trial: {
            "max_depth": trial.suggest_categorical('max_depth', [5, 10, 20, None]),
            "min_samples_split": trial.suggest_int('min_samples_split', 2, 10),
            "min_samples_leaf": trial.suggest_int('min_samples_leaf', 1, 4),
            "max_features": trial.suggest_categorical('max_features', [None, 'sqrt', 'log2'])
        }
    },
    "XGBoost": {
        "model": xgb.XGBRegressor,
        "params": lambda trial: {
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "n_estimators": trial.suggest_int("n_estimators", 100, 2000, step=100),
            "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.2),
            "subsample": trial.suggest_categorical("subsample", [0.7, 0.8, 1.0]),
            "colsample_bytree": trial.suggest_categorical("colsample_bytree", [0.7, 0.8, 1.0]),
        }
    },
    "RandomForest": {
        "model": RandomForestRegressor,
        "params": lambda trial: {
            "max_depth": trial.suggest_categorical('max_depth', [10, 20, 30]),
            "n_estimators": trial.suggest_int("n_estimators", 100, 500, step=100),
            "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 4),
            "bootstrap": trial.suggest_categorical("bootstrap", [True, False])
        }
    }
}

# TimeSeries Cross-validation strategy
tscv = TimeSeriesSplit(n_splits=5)

# Store results
results = {}

# Loop through models
for model_name, model_info in models.items():
    
    def objective(trial):
        params = model_info["params"](trial)
        model = model_info["model"](**params, random_state=42)

        mse_scores = []
        
        # Perform manual cross-validation using TimeSeriesSplit
        for train_index, val_index in tscv.split(X_train):
            X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
            y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

            model.fit(X_train_fold, y_train_fold)
            y_pred_fold = model.predict(X_val_fold)
            mse_scores.append(mean_squared_error(y_val_fold, y_pred_fold))

        return sum(mse_scores) / len(mse_scores)  # Return average MSE
    
    # Optimize hyperparameters using Optuna
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=50)

    best_params = study.best_params
    print(f"\nBest Hyperparameters for {model_name}: {best_params}\n")

    # Train the best model on the full training set
    best_model = model_info["model"](**best_params, random_state=42)
    best_model.fit(X_train, y_train)

    # Predict on test set
    y_pred_test = best_model.predict(X_test)
    mse_test = mean_squared_error(y_test, y_pred_test)

    # Save predictions to CSV
    pred_df = pd.DataFrame({"Actual": y_test, "Predicted": y_pred_test}, index=y_test.index)
    pred_csv_path = os.path.join(output_dir, f"{model_name}_predictions.csv")
    pred_df.to_csv(pred_csv_path, index=True)
    
    # Save results
    results[model_name] = {
        "Test MSE": mse_test,
        "Best Hyperparameters": best_params
    }

    print(f"{model_name} - Mean Squared Error (Test): {mse_test:.4f}")

    # Feature Importance Analysis
    if hasattr(best_model, "feature_importances_"):
        plt.figure(figsize=(12, 6))
        plt.barh(X_train.columns, best_model.feature_importances_, align='center')
        plt.xlabel("Feature Importance")
        plt.ylabel("Feature")
        plt.title(f"{model_name} Feature Importance Analysis")
        plt.show()

    # Plot Actual vs. Predicted Values
    plt.figure(figsize=(12, 6))
    plt.plot(y_test.index, y_test, label="Actual Predictions" , linewidth=1)
    plt.plot(y_test.index, y_pred_test, label=f"Predicted with ({model_name})", color='red' , linestyle='-' , linewidth=1)
    plt.legend()
    plt.xlabel("Time")
    plt.ylabel("Predictions")
    plt.title(f"{model_name} Time-Series Forecasting")
    plt.show()

# Save all results to JSON file
results_json_path = os.path.join(output_dir, "summary_results.json")
with open(results_json_path, "w") as f:
    json.dump(results, f, indent=4)

# Print final results
print("\nFinal Model Performance:")
for model, data in results.items():
    print(f"{model}: Test MSE = {data['Test MSE']:.4f}")


# KAN

In [ ]:
df = df.iloc[-2688:]

df_1=pd.read_csv(r'C:\Users\gkeep\Desktop\Results\DNN\KAN_11.03_n=100\KAN_final_predictions.csv') # Validation Predictions
df_2=pd.read_csv(r'C:\Users\gkeep\Desktop\Results\DNN\TEST\KAN_14.03_Test_Results\KAN_final_predictions.csv')         # Test predictions

# Concatenate both DataFrames vertically
new_df = pd.concat([df_1, df_2], ignore_index=True)

df["Target"] = new_df["KAN_unscaled"].values  # Ensure values are aligned

df=df.drop(columns=['WHE (kW)_+lag1440'])

DF_test = df.iloc[-1344:]

## Normal Split

In [ ]:
import os
import json
import optuna
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

# ----------------------
# User-defined paths
# ----------------------
output_dir = r"C:\Users\gkeep\Desktop\Results\Regressors\KAN\ElasticNet"
os.makedirs(output_dir, exist_ok=True)

# ----------------------
# Load or define your data
# ----------------------
# Replace with your actual data
# df = pd.read_csv("your_data.csv", index_col=0, parse_dates=[0])
# For example:
# df = pd.read_csv("my_dataset.csv")
# DF_test = pd.read_csv("my_testset.csv")

# Placeholder (make sure to replace!)
# df, DF_test must contain "Target" column
#assert 'df' in globals() and 'DF_test' in globals(), "Load your dataset as df and DF_test first."

X_train = df.drop(columns=['Target'])
y_train = df['Target']
X_test = DF_test.drop(columns=['Target'])
y_test = DF_test['Target']

# ----------------------
# Scaling
# ----------------------
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ----------------------
# Optuna Optimization
# ----------------------
def objective(trial):
    params = {
        "alpha": trial.suggest_loguniform("alpha", 1e-4, 10.0),
        "l1_ratio": trial.suggest_uniform("l1_ratio", 0.0, 1.0),
        "fit_intercept": trial.suggest_categorical("fit_intercept", [True, False])
    }

    model = ElasticNet(**params, max_iter=10000, random_state=42)
    tscv = TimeSeriesSplit(n_splits=5)
    mse_scores = []

    for train_idx, val_idx in tscv.split(X_train_scaled):
        X_tr, X_val = X_train_scaled[train_idx], X_train_scaled[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

        model.fit(X_tr, y_tr)
        preds = model.predict(X_val)
        mse_scores.append(mean_squared_error(y_val, preds))

    return np.mean(mse_scores)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

best_params = study.best_params
print("Best Parameters:", best_params)

# ----------------------
# Train on full training data
# ----------------------
final_model = ElasticNet(**best_params, max_iter=10000, random_state=42)
final_model.fit(X_train_scaled, y_train)
y_pred = final_model.predict(X_test_scaled)
test_mse = mean_squared_error(y_test, y_pred)
print(f"Test MSE: {test_mse:.4f}")

# ----------------------
# Save predictions
# ----------------------
pred_df = pd.DataFrame({"Actual": y_test, "Predicted": y_pred}, index=y_test.index)
pred_df.to_csv(os.path.join(output_dir, "ElasticNet_predictions.csv"))

# ----------------------
# Save results
# ----------------------
results = {
    "Test MSE": test_mse,
    "Best Hyperparameters": best_params
}
with open(os.path.join(output_dir, "ElasticNet_results.json"), "w") as f:
    json.dump(results, f, indent=4)

# ----------------------
# Feature Importance Plot
# ----------------------
plt.figure(figsize=(12, 6))
plt.barh(X_train.columns, np.abs(final_model.coef_))
plt.xlabel("Coefficient Magnitude")
plt.ylabel("Feature")
plt.title("ElasticNet Feature Importance")
plt.tight_layout()
plt.show()

# ----------------------
# Plot Predictions
# ----------------------
plt.figure(figsize=(12, 6))
plt.plot(y_test.index, y_test, label="Actual", linewidth=1)
plt.plot(y_test.index, y_pred, label="ElasticNet Prediction", color="red", linewidth=1)
plt.xlabel("Time")
plt.ylabel("Target")
plt.title("ElasticNet Forecast vs Actual")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import optuna
import lightgbm as lgb
import xgboost as xgb
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit

# Create output directory if not exists
output_dir = r"C:\Users\gkeep\Desktop\Results\Regressors\KAN"
os.makedirs(output_dir, exist_ok=True)

# Define training and test sets
X_train = df.drop(columns=['Target'])  # Train on Validation + Test
y_train = df['Target']

X_test = DF_test.drop(columns=['Target'])  # Predict on Test
y_test = DF_test['Target']

# Define models and their hyperparameter search spaces
models = {
    "LightGBM": {
        "model": lgb.LGBMRegressor,
        "params": lambda trial: {
            "num_leaves": trial.suggest_int('num_leaves', 31, 100),
            "learning_rate": trial.suggest_loguniform('learning_rate', 0.01, 0.1),
            "n_estimators": trial.suggest_int('n_estimators', 100, 1000),
            "max_depth": trial.suggest_categorical('max_depth', [-1, 10, 20])
        }
    },
    "DecisionTree": {
        "model": DecisionTreeRegressor,
        "params": lambda trial: {
            "max_depth": trial.suggest_categorical('max_depth', [5, 10, 20, None]),
            "min_samples_split": trial.suggest_int('min_samples_split', 2, 10),
            "min_samples_leaf": trial.suggest_int('min_samples_leaf', 1, 4),
            "max_features": trial.suggest_categorical('max_features', [None, 'sqrt', 'log2'])
        }
    },
    "XGBoost": {
        "model": xgb.XGBRegressor,
        "params": lambda trial: {
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "n_estimators": trial.suggest_int("n_estimators", 100, 2000, step=100),
            "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.2),
            "subsample": trial.suggest_categorical("subsample", [0.7, 0.8, 1.0]),
            "colsample_bytree": trial.suggest_categorical("colsample_bytree", [0.7, 0.8, 1.0]),
        }
    },
    "RandomForest": {
        "model": RandomForestRegressor,
        "params": lambda trial: {
            "max_depth": trial.suggest_categorical('max_depth', [10, 20, 30]),
            "n_estimators": trial.suggest_int("n_estimators", 100, 500, step=100),
            "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 4),
            "bootstrap": trial.suggest_categorical("bootstrap", [True, False])
        }
    }
}

# TimeSeries Cross-validation strategy
tscv = TimeSeriesSplit(n_splits=5)

# Store results
results = {}

# Loop through models
for model_name, model_info in models.items():
    
    def objective(trial):
        params = model_info["params"](trial)
        model = model_info["model"](**params, random_state=42)

        mse_scores = []
        
        # Perform manual cross-validation using TimeSeriesSplit
        for train_index, val_index in tscv.split(X_train):
            X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
            y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

            model.fit(X_train_fold, y_train_fold)
            y_pred_fold = model.predict(X_val_fold)
            mse_scores.append(mean_squared_error(y_val_fold, y_pred_fold))

        return sum(mse_scores) / len(mse_scores)  # Return average MSE
    
    # Optimize hyperparameters using Optuna
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=50)

    best_params = study.best_params
    print(f"\nBest Hyperparameters for {model_name}: {best_params}\n")

    # Train the best model on the full training set
    best_model = model_info["model"](**best_params, random_state=42)
    best_model.fit(X_train, y_train)

    # Predict on test set
    y_pred_test = best_model.predict(X_test)
    mse_test = mean_squared_error(y_test, y_pred_test)

    # Save predictions to CSV
    pred_df = pd.DataFrame({"Actual": y_test, "Predicted": y_pred_test}, index=y_test.index)
    pred_csv_path = os.path.join(output_dir, f"{model_name}_predictions.csv")
    pred_df.to_csv(pred_csv_path, index=True)
    
    # Save results
    results[model_name] = {
        "Test MSE": mse_test,
        "Best Hyperparameters": best_params
    }

    print(f"{model_name} - Mean Squared Error (Test): {mse_test:.4f}")

    # Feature Importance Analysis
    if hasattr(best_model, "feature_importances_"):
        plt.figure(figsize=(12, 6))
        plt.barh(X_train.columns, best_model.feature_importances_, align='center')
        plt.xlabel("Feature Importance")
        plt.ylabel("Feature")
        plt.title(f"{model_name} Feature Importance Analysis")
        plt.show()

    # Plot Actual vs. Predicted Values
    plt.figure(figsize=(12, 6))
    plt.plot(y_test.index, y_test, label="Actual Predictions" , linewidth=1)
    plt.plot(y_test.index, y_pred_test, label=f"Predicted with ({model_name})", color='red' , linestyle='-' , linewidth=1)
    plt.legend()
    plt.xlabel("Time")
    plt.ylabel("Predictions")
    plt.title(f"{model_name} Time-Series Forecasting")
    plt.show()

# Save all results to JSON file
results_json_path = os.path.join(output_dir, "summary_results.json")
with open(results_json_path, "w") as f:
    json.dump(results, f, indent=4)

# Print final results
print("\nFinal Model Performance:")
for model, data in results.items():
    print(f"{model}: Test MSE = {data['Test MSE']:.4f}")


# Plot

In [ ]:
# Your existing imports
import os
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Define base directory
base_dir = r"C:\Users\gkeep\Desktop\Results\Regressors"

# List of neural network folders
neural_networks = ["TCN", "LSTM", "KAN", "TSMixerx", "NBEATSx"]

# Initialize empty dictionary
mse_data = {}

# Load JSON MSEs from each folder
for nn_name in neural_networks:
    json_path = os.path.join(base_dir, nn_name, "summary_results.json")
    if os.path.exists(json_path):
        with open(json_path, "r") as f:
            results = json.load(f)
        for model, data in results.items():
            if model not in mse_data:
                mse_data[model] = {}
            mse_data[model][nn_name] = data["Test MSE"]

# 🔹 Manually insert ElasticNet results
mse_data["ElasticNet"] = {
    "LSTM": 0.0605757647248136,
    "TCN": 0.049132745019164016,
    "TSMixerx": 0.0509752464354896,
    "NBEATSx": 0.07779817207695296,
    "KAN": 0.0638408730398479
}

# Convert to DataFrame
mse_df = pd.DataFrame(mse_data).T

# Plot Heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(mse_df, annot=True, cmap="coolwarm", fmt=".3f", linewidths=0.5)
plt.xlabel("Neural Networks")
plt.ylabel("Surrogate Models")
plt.title("MSE Heatmap: Surrogate Models vs. Neural Networks")
plt.tight_layout()
# Save the figure before showing it
plt.savefig(
    r'C:\Users\gkeep\Desktop\Results\Regressors\Surrogate_Heatmap.png',  # Change path + name
    dpi=300,             # High resolution (300 dots per inch)
    bbox_inches='tight'  # Trim extra whitespace
)
plt.show()
